What i want to do: 
- read in postcode dataset
- loop through the code point files 
- join postcodes in each file 
- make sure these are saved and each time loop through a file it doesn't delete earlier merges. 

In [3]:
# Set libraries
import pandas as pd
import glob
import os
import numpy as np
import datetime

In [5]:
# Read in postcode dataset
SWW_postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/SWW postcodes.csv')

In [21]:
SWW_postcode = SWW_postcode.set_index('postcode')

KeyError: 'postcode'

In [22]:
SWW_postcode

,date
postcode,
EX12TW,21/08/2014
BH25 5LA,24/02/2017
DT7 3AD,05/05/2011
DT7 3BB,16/01/2013
DT7 3BH,21/10/2011
DT7 3BJ,08/07/2016
DT7 3BP,13/02/2014
DT7 3DU,22/09/2011
DT7 3EZ,06/09/2013


In [8]:
files = glob.glob(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/*')

In [9]:
files

['/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ab.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/al.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/b.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ba.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bb.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bd.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bh.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bl.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bn.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/br.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bs.csv',


In [12]:
for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file))
    print('Processed {fname}'.format(fname=file))

Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ab.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/al.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/b.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ba.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bb.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bd.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bh.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bl.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bn.csv
Processed /Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/br.csv
Processed /Users/annalisasheeha

In [36]:
postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/dt.csv', header = None)

In [37]:
postcode

,0,1,2,3,4,5,6,7,8,9
0,DT1 1AA,10,368730,90722,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
1,DT1 1AB,10,369874,90687,E92000001,E19000002,E18000010,E10000009,E07000052,E05010589
2,DT1 1AD,10,369245,90319,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
3,DT1 1AE,10,369076,90500,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
4,DT1 1AF,10,369092,90468,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
5,DT1 1AG,10,369069,90409,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
6,DT1 1AH,10,368593,90782,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
7,DT1 1AL,10,370090,90402,E92000001,E19000002,E18000010,E10000009,E07000052,E05010589
8,DT1 1AN,10,369956,90295,E92000001,E19000002,E18000010,E10000009,E07000052,E05010589
9,DT1 1AP,10,369204,90495,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590


In [79]:
labels = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Doc/Code-Point_Open_Column_Headers.csv', header = None)

In [80]:
labels

,0,1,2,3,4,5,6,7,8,9
0,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code


In [82]:
labels = labels.set_index([0])

In [83]:
labels

,1,2,3,4,5,6,7,8,9
0,,,,,,,,,
Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code


In [84]:
postcode = postcode.set_index([0])

In [91]:
data = pd.concat([labels,postcode])

In [94]:
data

Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
0,,,,,,,,,
Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
DT1 1AA,10,368730,90722,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AB,10,369874,90687,E92000001,E19000002,E18000010,E10000009,E07000052,E05010589
DT1 1AD,10,369245,90319,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AE,10,369076,90500,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AF,10,369092,90468,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AG,10,369069,90409,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AH,10,368593,90782,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AL,10,370090,90402,E92000001,E19000002,E18000010,E10000009,E07000052,E05010589


In [93]:
data.columns = data.iloc[0]

In [97]:
data.reindex(data.index.drop('Postcode'))

Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
0,,,,,,,,,
DT1 1AA,10,368730,90722,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AB,10,369874,90687,E92000001,E19000002,E18000010,E10000009,E07000052,E05010589
DT1 1AD,10,369245,90319,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AE,10,369076,90500,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AF,10,369092,90468,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AG,10,369069,90409,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AH,10,368593,90782,E92000001,E19000002,E18000010,E10000009,E07000052,E05010590
DT1 1AL,10,370090,90402,E92000001,E19000002,E18000010,E10000009,E07000052,E05010589
DT1 1AN,10,369956,90295,E92000001,E19000002,E18000010,E10000009,E07000052,E05010589
